# Métricas de Evaluación de Modelos de Clasificación

Este notebook cubre:
- Métricas de evaluación para modelos de clasificación
- Análisis detallado de las métricas generadas por los modelos
- Visualización de métricas
- Interpretación de resultados

## 1. Importación de Librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report,
    roc_curve, auc, roc_auc_score,
    precision_recall_curve, average_precision_score
)
from sklearn.preprocessing import label_binarize
from itertools import cycle
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Librerías importadas correctamente")

Librerías importadas correctamente


## 2. Cargar Modelos y Predicciones del Notebook Anterior

**Nota:** Ejecuta primero el notebook `02_modelos_clasificacion.ipynb` para tener los modelos entrenados.

In [2]:
# Si ejecutaste el notebook anterior, estos objetos deberían estar disponibles
# Si no, descomenta y ejecuta el código siguiente para entrenar modelos rápidamente

# Código de ejemplo para cargar datos y entrenar un modelo rápido
# (Descomenta si no ejecutaste el notebook anterior)
"""
import mysql.connector
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

DB_CONFIG = {
    'host': '127.0.0.1',
    'database': 'palantir_maintenance',
    'user': 'root',
    'password': 'admin',
    'port': 3306
}

connection = mysql.connector.connect(**DB_CONFIG)
df = pd.read_sql("SELECT * FROM faliure_probability_base", connection)
connection.close()

def crear_clase_riesgo(row):
    failure_count = row.get('failure_count_365d', 0) or 0
    sensor_critical = row.get('sensor_critical_count_30d', 0) or 0
    if failure_count >= 5 or sensor_critical >= 10:
        return 3
    elif failure_count >= 3 or sensor_critical >= 5:
        return 2
    elif failure_count >= 1 or sensor_critical >= 2:
        return 1
    else:
        return 0

df['clase_riesgo'] = df.apply(crear_clase_riesgo, axis=1)

feature_columns = [
    'asset_age_days', 'sensor_total_readings_30d', 'sensor_warning_count_30d',
    'sensor_critical_count_30d', 'failure_count_365d', 'failure_critical_count',
    'task_total_365d', 'order_total_365d'
]

available_features = [col for col in feature_columns if col in df.columns]
X = df[available_features].fillna(0)
y = df['clase_riesgo']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

dt_model = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
"""

print("Asegúrate de haber ejecutado el notebook anterior o descomenta el código de arriba")

Asegúrate de haber ejecutado el notebook anterior o descomenta el código de arriba


## 3. Métricas Básicas de Clasificación

### 3.1 Accuracy, Precision, Recall, F1-Score

In [3]:
def calcular_metricas_basicas(y_true, y_pred, nombre_modelo="Modelo"):
    """Calcular métricas básicas de clasificación"""
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    metricas = {
        'Modelo': nombre_modelo,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    }
    
    print(f"\n=== Métricas para {nombre_modelo} ===")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    
    return metricas

# Calcular métricas para los modelos (si están disponibles)
if 'y_test' in locals() and 'y_pred_dt' in locals():
    metricas_dt = calcular_metricas_basicas(y_test, y_pred_dt, "Árbol de Decisión")
    
if 'y_test' in locals() and 'y_pred_rf' in locals():
    metricas_rf = calcular_metricas_basicas(y_test, y_pred_rf, "Random Forest")
else:
    print("Ejecuta primero el notebook de modelos de clasificación")

Ejecuta primero el notebook de modelos de clasificación


### 3.2 Explicación de las Métricas

**Accuracy (Precisión):** Proporción de predicciones correctas sobre el total.
- Fórmula: (TP + TN) / (TP + TN + FP + FN)
- Útil cuando las clases están balanceadas

**Precision (Precisión por clase):** Proporción de predicciones positivas que son correctas.
- Fórmula: TP / (TP + FP)
- Mide cuántas de las predicciones positivas son realmente positivas

**Recall (Sensibilidad):** Proporción de casos positivos que fueron correctamente identificados.
- Fórmula: TP / (TP + FN)
- Mide cuántos casos positivos fueron capturados

**F1-Score:** Media armónica de Precision y Recall.
- Fórmula: 2 * (Precision * Recall) / (Precision + Recall)
- Balance entre Precision y Recall

## 4. Matriz de Confusión Detallada

In [4]:
def plot_confusion_matrix_detallada(y_true, y_pred, nombre_modelo, clases_nombres):
    """Visualizar matriz de confusión con métricas adicionales"""
    cm = confusion_matrix(y_true, y_pred)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Matriz de confusión normalizada
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Gráfico 1: Matriz absoluta
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
                xticklabels=clases_nombres, yticklabels=clases_nombres)
    axes[0].set_title(f'Matriz de Confusión - {nombre_modelo} (Absoluta)')
    axes[0].set_ylabel('Verdadero')
    axes[0].set_xlabel('Predicho')
    
    # Gráfico 2: Matriz normalizada
    sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues', ax=axes[1],
                xticklabels=clases_nombres, yticklabels=clases_nombres)
    axes[1].set_title(f'Matriz de Confusión - {nombre_modelo} (Normalizada)')
    axes[1].set_ylabel('Verdadero')
    axes[1].set_xlabel('Predicho')
    
    plt.tight_layout()
    plt.show()
    
    # Calcular métricas por clase
    print(f"\nMétricas por clase para {nombre_modelo}:")
    for i, clase_nombre in enumerate(clases_nombres):
        tp = cm[i, i]
        fp = cm[:, i].sum() - tp
        fn = cm[i, :].sum() - tp
        tn = cm.sum() - (tp + fp + fn)
        
        precision_clase = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall_clase = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_clase = 2 * (precision_clase * recall_clase) / (precision_clase + recall_clase) if (precision_clase + recall_clase) > 0 else 0
        
        print(f"\n{clase_nombre}:")
        print(f"  Precision: {precision_clase:.4f}")
        print(f"  Recall:    {recall_clase:.4f}")
        print(f"  F1-Score:  {f1_clase:.4f}")

# Visualizar matrices de confusión
clases_nombres = ['Bajo', 'Medio', 'Alto', 'Crítico']

if 'y_test' in locals() and 'y_pred_dt' in locals():
    plot_confusion_matrix_detallada(y_test, y_pred_dt, "Árbol de Decisión", clases_nombres)
    
if 'y_test' in locals() and 'y_pred_rf' in locals():
    plot_confusion_matrix_detallada(y_test, y_pred_rf, "Random Forest", clases_nombres)

## 5. Classification Report Detallado

In [5]:
def analizar_classification_report(y_true, y_pred, nombre_modelo, clases_nombres):
    """Analizar el reporte de clasificación"""
    print(f"\n{'='*60}")
    print(f"Classification Report - {nombre_modelo}")
    print(f"{'='*60}")
    report = classification_report(y_true, y_pred, target_names=clases_nombres, output_dict=True)
    
    # Convertir a DataFrame para mejor visualización
    df_report = pd.DataFrame(report).transpose()
    print(df_report.round(4))
    
    # Visualización
    fig, ax = plt.subplots(figsize=(10, 6))
    df_report_plot = df_report.iloc[:-3, :3]  # Excluir 'accuracy', 'macro avg', 'weighted avg'
    df_report_plot.plot(kind='bar', ax=ax)
    plt.title(f'Métricas por Clase - {nombre_modelo}')
    plt.ylabel('Score')
    plt.xlabel('Clase')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Métricas')
    plt.tight_layout()
    plt.show()
    
    return df_report

# Analizar reportes
if 'y_test' in locals() and 'y_pred_dt' in locals():
    report_dt = analizar_classification_report(y_test, y_pred_dt, "Árbol de Decisión", clases_nombres)
    
if 'y_test' in locals() and 'y_pred_rf' in locals():
    report_rf = analizar_classification_report(y_test, y_pred_rf, "Random Forest", clases_nombres)

## 6. Curvas ROC (Receiver Operating Characteristic)

Para problemas multiclase, usamos One-vs-Rest (OvR)

In [6]:
def plot_roc_curves_multiclass(y_true, y_pred_proba, nombre_modelo, clases_nombres):
    """Plot ROC curves para clasificación multiclase"""
    # Binarizar las etiquetas
    y_true_bin = label_binarize(y_true, classes=range(len(clases_nombres)))
    n_classes = len(clases_nombres)
    
    # Calcular ROC para cada clase
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    # Calcular micro-average ROC
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_pred_proba.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    # Plot
    plt.figure(figsize=(10, 8))
    
    # Plot para cada clase
    colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                label=f'ROC {clases_nombres[i]} (AUC = {roc_auc[i]:.2f})')
    
    # Plot micro-average
    plt.plot(fpr["micro"], tpr["micro"], color='deeppink', linestyle='--', lw=2,
            label=f'ROC micro-average (AUC = {roc_auc["micro"]:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curves - {nombre_modelo}')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return roc_auc

# Obtener probabilidades de predicción (si el modelo las soporta)
if 'dt_model' in locals() and 'X_test' in locals():
    try:
        y_pred_proba_dt = dt_model.predict_proba(X_test)
        roc_auc_dt = plot_roc_curves_multiclass(y_test, y_pred_proba_dt, "Árbol de Decisión", clases_nombres)
        print(f"\nAUC scores - Árbol de Decisión: {roc_auc_dt}")
    except Exception as e:
        print(f"No se pudieron calcular las curvas ROC: {e}")
        
if 'rf_model' in locals() and 'X_test' in locals():
    try:
        y_pred_proba_rf = rf_model.predict_proba(X_test)
        roc_auc_rf = plot_roc_curves_multiclass(y_test, y_pred_proba_rf, "Random Forest", clases_nombres)
        print(f"\nAUC scores - Random Forest: {roc_auc_rf}")
    except Exception as e:
        print(f"No se pudieron calcular las curvas ROC: {e}")

## 7. Curvas Precision-Recall

In [7]:
def plot_precision_recall_curves(y_true, y_pred_proba, nombre_modelo, clases_nombres):
    """Plot Precision-Recall curves para clasificación multiclase"""
    y_true_bin = label_binarize(y_true, classes=range(len(clases_nombres)))
    n_classes = len(clases_nombres)
    
    precision = dict()
    recall = dict()
    average_precision = dict()
    
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_pred_proba[:, i])
        average_precision[i] = average_precision_score(y_true_bin[:, i], y_pred_proba[:, i])
    
    # Micro-average
    precision["micro"], recall["micro"], _ = precision_recall_curve(
        y_true_bin.ravel(), y_pred_proba.ravel())
    average_precision["micro"] = average_precision_score(y_true_bin, y_pred_proba, average="micro")
    
    # Plot
    plt.figure(figsize=(10, 8))
    
    colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(recall[i], precision[i], color=color, lw=2,
                label=f'PR {clases_nombres[i]} (AP = {average_precision[i]:.2f})')
    
    plt.plot(recall["micro"], precision["micro"], color='deeppink', linestyle='--', lw=2,
            label=f'PR micro-average (AP = {average_precision["micro"]:.2f})')
    
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curves - {nombre_modelo}')
    plt.legend(loc="lower left")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return average_precision

# Plot Precision-Recall curves
if 'y_test' in locals() and 'dt_model' in locals() and 'X_test' in locals():
    try:
        y_pred_proba_dt = dt_model.predict_proba(X_test)
        ap_dt = plot_precision_recall_curves(y_test, y_pred_proba_dt, "Árbol de Decisión", clases_nombres)
        print(f"\nAverage Precision - Árbol de Decisión: {ap_dt}")
    except Exception as e:
        print(f"Error: {e}")
        
if 'y_test' in locals() and 'rf_model' in locals() and 'X_test' in locals():
    try:
        y_pred_proba_rf = rf_model.predict_proba(X_test)
        ap_rf = plot_precision_recall_curves(y_test, y_pred_proba_rf, "Random Forest", clases_nombres)
        print(f"\nAverage Precision - Random Forest: {ap_rf}")
    except Exception as e:
        print(f"Error: {e}")

## 8. Comparación Completa de Métricas entre Modelos

In [8]:
def comparar_modelos_completo(y_test, modelos_dict, clases_nombres):
    """Comparar todos los modelos con todas las métricas"""
    resultados = []
    
    for nombre, (y_pred, modelo) in modelos_dict.items():
        # Métricas básicas
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        
        # AUC (si el modelo tiene predict_proba)
        try:
            y_pred_proba = modelo.predict_proba(y_test.values.reshape(-1, 1) if hasattr(y_test, 'values') else y_test)
            y_test_bin = label_binarize(y_test, classes=range(len(clases_nombres)))
            auc_score = roc_auc_score(y_test_bin, y_pred_proba, average='weighted', multi_class='ovr')
        except:
            auc_score = None
        
        resultados.append({
            'Modelo': nombre,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'AUC': auc_score
        })
    
    df_comparacion = pd.DataFrame(resultados)
    
    # Visualización
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))
    
    # Gráfico 1: Métricas básicas
    df_comparacion_plot = df_comparacion.set_index('Modelo')[['Accuracy', 'Precision', 'Recall', 'F1-Score']]
    df_comparacion_plot.plot(kind='bar', ax=axes[0])
    axes[0].set_title('Comparación de Métricas Básicas')
    axes[0].set_ylabel('Score')
    axes[0].legend(title='Métricas')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Gráfico 2: Tabla de resultados
    axes[1].axis('tight')
    axes[1].axis('off')
    tabla = axes[1].table(cellText=df_comparacion.round(4).values,
                         colLabels=df_comparacion.columns,
                         cellLoc='center',
                         loc='center')
    tabla.auto_set_font_size(False)
    tabla.set_fontsize(10)
    tabla.scale(1.2, 1.5)
    
    plt.tight_layout()
    plt.show()
    
    return df_comparacion

# Comparar modelos
if 'y_test' in locals() and 'y_pred_dt' in locals() and 'y_pred_rf' in locals():
    modelos_dict = {
        'Árbol de Decisión': (y_pred_dt, dt_model if 'dt_model' in locals() else None),
        'Random Forest': (y_pred_rf, rf_model if 'rf_model' in locals() else None)
    }
    
    # Filtrar modelos válidos
    modelos_dict = {k: v for k, v in modelos_dict.items() if v[1] is not None}
    
    if modelos_dict:
        df_comparacion = comparar_modelos_completo(y_test, modelos_dict, clases_nombres)
        print("\nComparación de Modelos:")
        print(df_comparacion.round(4))
else:
    print("No hay suficientes modelos para comparar")

No hay suficientes modelos para comparar


## Resumen

En este notebook hemos aprendido:
1. ✅ Métricas básicas de clasificación (Accuracy, Precision, Recall, F1-Score)
2. ✅ Análisis detallado de matrices de confusión
3. ✅ Classification reports por clase
4. ✅ Curvas ROC para problemas multiclase
5. ✅ Curvas Precision-Recall
6. ✅ Comparación completa de modelos

**Próximos pasos:** En el siguiente notebook aprenderemos sobre LightGBM.